# Example Workflow for CM2.6

- Interpolate atmosphere onto ocean
- Recalculate flux terms from ocean res (`full_res_*`)
- Coarsen the recalculated flux + flux input fields
- Recompute flux terms once again from coarsened input fields (`coarse_res_*`)
- Look at the difference (`full_res_* - coarse_res_*`)

In [1]:
# !mamba install aerobulk-python -y

In [2]:
from aerobulk import skin, noskin
import fsspec
import xarray as xr
import numpy as np
import xesmf as xe
import os
from intake import open_catalog
import matplotlib.pyplot as plt

In [3]:
kwargs = dict(consolidated=True, use_cftime=True)
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/GFDL_CM2.6.yaml")
ds_ocean  = cat["GFDL_CM2_6_control_ocean_surface"].to_dask()
ds_flux  = cat["GFDL_CM2_6_control_ocean_boundary_flux"].to_dask()
# xarray says not to do this
# ds_atmos = xr.open_zarr('gs://cmip6/GFDL_CM2_6/control/atmos_daily.zarr', chunks={'time':1}, **kwargs)
ds_atmos = xr.open_zarr('gs://cmip6/GFDL_CM2_6/control/atmos_daily.zarr', **kwargs)
ds_oc_grid  = cat["GFDL_CM2_6_grid"].to_dask()
# cut to same time
all_dims = set(list(ds_ocean.dims)+list(ds_atmos.dims))
ds_ocean, ds_atmos = xr.align(
    ds_ocean,
    ds_atmos,
    join='inner',
    exclude=(di for di in all_dims if di !='time')
)


# instead do this
ds_atmos = ds_atmos.chunk({'time':1})

In [4]:
import json
import gcsfs

# 👇 replace with your key 
with open('/home/jovyan/keys/pangeo-forge-ocean-transport-4967-347e2048c5a1.json') as token_file:
    token = json.load(token_file)
fs = gcsfs.GCSFileSystem(token=token)
path = 'ocean-transport-group/scale-aware-air-sea/regridding_weights/CM26_atmos2ocean.zarr'
mapper = fs.get_mapper(path)
ds_regridder = xr.open_zarr(mapper).load()
regridder = xe.Regridder(
    ds_atmos.olr.to_dataset(name='dummy').isel(time=0),
    ds_ocean.surface_temp.to_dataset(name='dummy').isel(time=0),
    'bilinear',
    weights=ds_regridder,
    periodic=True
)
regridder

xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_360x576_2700x3600_peri.nc 
Reuse pre-computed weights? False 
Input grid shape:           (360, 576) 
Output grid shape:          (2700, 3600) 
Periodic in longitude?      True

In [5]:
ds_atmos_regridded = regridder(ds_atmos[['slp', 'v_ref', 'u_ref', 't_ref', 'q_ref', 'swdn_sfc', 'lwdn_sfc']])
ds_atmos_regridded

## combine into merged dataset
ds_merged = xr.merge(
    [
        ds_atmos_regridded,
        ds_ocean[['surface_temp']],
    ]
)
ds_merged = ds_merged.transpose(
    'xt_ocean', 'yt_ocean', 'time'
)

In [6]:
from distributed import LocalCluster, Client
import dask
dask.config.set(**{"distributed.dashboard.link": "/user/{JUPYTERHUB_USER}/proxy/{port}/status"})
# cluster = LocalCluster(n_workers=4, threads_per_worker=1)
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/jbusecke/proxy/8787/status,
Dashboard: /user/jbusecke/proxy/8787/status,Workers: 4
Total threads: 16,Total memory: 58.88 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44147,Workers: 4
Dashboard: /user/jbusecke/proxy/8787/status,Total threads: 16
Started: Just now,Total memory: 58.88 GiB
Comm: tcp://127.0.0.1:46863,Total threads: 4
Dashboard: /user/jbusecke/proxy/45733/status,Memory: 14.72 GiB
Nanny: tcp://127.0.0.1:46837,


### Cluster notes
This seems to work fine if each worker has ~12 GB, for smaller ones thsi produces all kind of warnings, and dying workers

In [7]:
# Ok lets try this with dask (this looks absolutely horrible for small workers)
# Now seems to run ok (even though both the regridding, and especially the aerobulk calls take a long time)
ds = ds_merged.isel(time=slice(0,14))
ds

<xarray.Dataset>
Dimensions:       (yt_ocean: 2700, xt_ocean: 3600, time: 14)
Coordinates:
  * yt_ocean      (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
  * xt_ocean      (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * time          (time) object 0181-01-01 12:00:00 ... 0181-01-14 12:00:00
Data variables:
    slp           (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(3600, 2700, 1), meta=np.ndarray>
    v_ref         (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(3600, 2700, 1), meta=np.ndarray>
    u_ref         (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(3600, 2700, 1), meta=np.ndarray>
    t_ref         (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(3600, 2700, 1), meta=np.ndarray>
    q_ref         (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(3600, 2700, 1), meta=np.ndarray>
    swdn_sfc      (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(3600, 2700, 1), meta=np.ndarray>
    lwdn_sfc      (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(3600, 2700, 1), meta=np.ndarray>
    surface_temp  (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(3600, 2700, 1), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [8]:
ds.nbytes/1e9

4.354610512

In [9]:
# for initial testing due to dask problems. Load the whole dataset into memory
ds = ds.persist()

In [10]:
ds

<xarray.Dataset>
Dimensions:       (yt_ocean: 2700, xt_ocean: 3600, time: 14)
Coordinates:
  * yt_ocean      (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
  * xt_ocean      (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * time          (time) object 0181-01-01 12:00:00 ... 0181-01-14 12:00:00
Data variables:
    slp           (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(3600, 2700, 1), meta=np.ndarray>
    v_ref         (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(3600, 2700, 1), meta=np.ndarray>
    u_ref         (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(3600, 2700, 1), meta=np.ndarray>
    t_ref         (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(3600, 2700, 1), meta=np.ndarray>
    q_ref         (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(3600, 2700, 1), meta=np.ndarray>
    swdn_sfc      (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(3600, 2700, 1), meta=np.ndarray>
    lwdn_sfc      (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(3600, 2700, 1), meta=np.ndarray>
    surface_temp  (xt_ocean, yt_ocean, time) float32 dask.array<chunksize=(3600, 2700, 1), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [11]:
def noskin_ds_wrapper(ds_in):
    ds_out = xr.Dataset()
    ds_in = ds_in.copy(deep=False)
    
    sst = ds_in.surface_temp + 273.15
    t_zt = ds_in.t_ref
    hum_zt = ds_in.q_ref
    u_zu = ds_in.u_ref
    v_zu = ds_in.v_ref
    slp = ds_in.slp * 100 # check this
    zu = 10
    zt = 2
    
    ql, qh, taux, tauy, evap =  noskin(
        sst,
        t_zt,
        hum_zt,
        u_zu,
        v_zu,
        slp=slp,
        algo='ecmwf',
        zt=2,
        zu=10,
    )
    ds_out['ql'] = ql
    ds_out['qh'] = qh
    ds_out['evap'] = evap
    ds_out['taux'] = taux 
    ds_out['tauy'] = tauy
    return ds_out

In [12]:
ds_full_res = noskin_ds_wrapper(ds)

In [13]:
# ds_full_res['qh'].isel(time=slice(0,12)).mean('time').plot(x='xt_ocean', robust=True)

In [14]:
for var in ['qh', 'ql', 'evap']:
    ds[var] = ds_full_res[var]

In [15]:
# add area 
ds = ds.assign_coords(area=ds_oc_grid.area_t.load())

### Manually do a weighted coarsen
> Would be nice if this could work with the xr.weighted logic!

In [16]:
# # which fields contain nans?
# for n,da in ds.items():
#     print(n, np.isnan(da.isel(time=0)).any().load().data)

In [17]:
coarsen_win = dict(xt_ocean=20, yt_ocean=20)
mean_dims = ['xt_ocean', 'yt_ocean']

ds_weighted = ds * ds.area
ds_coarsened = ds_weighted.coarsen(**coarsen_win).mean(mean_dims) / ds.area.coarsen(**coarsen_win).mean(mean_dims)

# ds_weighted = ds * ds.area
# coarsen_win = dict(xt_ocean=20, yt_ocean=20)
# ds_coarsened = ds.coarsen(**coarsen_win).mean(mean_dims)


# Apply a new (strict) landmask to fields that had nans in them before
coarsened_landmask = coarsened_landmask = np.isnan(ds['surface_temp'].isel(time=0)).coarsen(**coarsen_win).sum(mean_dims)>0
for var in ['qh', 'ql', 'surface_temp', 't_ref', 'slp']:
    ds_coarsened[var] = ds_coarsened[var].where(~coarsened_landmask)
    
# # t_ref is weird. It has 0 values to begin with
# ds_coarsened['t_ref'] = ds_coarsened['t_ref'].fillna(275.0) #TODO: check if we can fill with 0
# ds_coarsened['slp'] = ds_coarsened['slp'].fillna(100000/100)

# # ad hoc filling of values
for va, fillval in [('q_ref', 0), ('u_ref', 0), ('v_ref', 0), ('slp', 100000/100), ('t_ref',275.0)]:
    ds_coarsened[va] = ds_coarsened[va].fillna(fillval)

In [18]:
ds_coarsened = ds_coarsened.load()

 
                    ----- AeroBulk_init -----
 
     *** Bulk parameterization to be used => "ecmwf"
     *** Cool-skin & Warm-layer schemes will NOT be used!
     *** Computational domain shape: Ni x Nj = 03600 x 02700
     *** Number of time records that will be treated:           1
     *** Number of iterations in bulk algos: nb_iter  =    6
     *** Filling the `mask` array...
         ==> number of points to mask:      2980707  (out of      9720000 )
     *** Type of prescribed air humidity  `specific humidity [kg/kg]`
 
                    ----- AeroBulk_init -----
 
     *** Bulk parameterization to be used => "ecmwf"
     *** Cool-skin & Warm-layer schemes will NOT be used!
     *** Computational domain shape: Ni x Nj = 03600 x 02700
     *** Number of time records that will be treated:           1
     *** Number of iterations in bulk algos: nb_iter  =    6
     *** Filling the `mask` array...
 
                    ----- AeroBulk_init -----
 
     *** Bulk parameterization t

2022-06-16 20:41:05,215 - distributed.worker - ERROR - failed during get data with tcp://127.0.0.1:33151 -> tcp://127.0.0.1:46863
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/tornado/iostream.py", line 867, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/tornado/iostream.py", line 1140, in read_from_fd
    return self.socket.recv_into(buf, len(buf))
TimeoutError: [Errno 110] Connection timed out

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/worker.py", line 1701, in get_data
    response = await comm.read(deserializers=serializers)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/comm/tcp.py", line 242, in read
    convert_stream_closed_error(self, e)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages

                    ----- AeroBulk_bye -----
 


2022-06-16 20:41:21,032 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:46863
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/tornado/iostream.py", line 867, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/tornado/iostream.py", line 1140, in read_from_fd
    return self.socket.recv_into(buf, len(buf))
ConnectionResetError: [Errno 104] Connection reset by peer

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/worker.py", line 3274, in gather_dep
    response = await get_data_from_worker(
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/worker.py", line 4590, in get_data_from_worker
    return await retry_operation(_get_data, operation="get_data_from_worker")
  File "/srv/conda/en

                    ----- AeroBulk_bye -----
 
                    ----- AeroBulk_bye -----
 
 
                    ----- AeroBulk_init -----
 
     *** Bulk parameterization to be used => "ecmwf"
     *** Cool-skin & Warm-layer schemes will NOT be used!
     *** Computational domain shape: Ni x Nj = 03600 x 02700
     *** Number of time records that will be treated:           1
     *** Number of iterations in bulk algos: nb_iter  =    6
     *** Filling the `mask` array...


...

2022-06-16 20:45:49,378 - distributed.worker - ERROR - failed during get data with tcp://127.0.0.1:46573 -> tcp://127.0.0.1:33151
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/tornado/iostream.py", line 867, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/tornado/iostream.py", line 1140, in read_from_fd
    return self.socket.recv_into(buf, len(buf))
TimeoutError: [Errno 110] Connection timed out

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/worker.py", line 1701, in get_data
    response = await comm.read(deserializers=serializers)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/comm/tcp.py", line 242, in read
    convert_stream_closed_error(self, e)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages

                    ----- AeroBulk_bye -----
 


2022-06-16 20:46:05,439 - distributed.worker - ERROR - Unable to allocate 4.04 EiB for an array with shape (4661802865091336519,) and data type uint8
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/worker.py", line 3274, in gather_dep
    response = await get_data_from_worker(
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/worker.py", line 4590, in get_data_from_worker
    return await retry_operation(_get_data, operation="get_data_from_worker")
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/utils_comm.py", line 381, in retry_operation
    return await retry(
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/utils_comm.py", line 366, in retry
    return await coro()
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/worker.py", line 4570, in _get_data
    response = await send_recv(
  File "/srv/conda/envs/notebook/lib/python3.9/si

 
                    ----- AeroBulk_init -----
 
     *** Bulk parameterization to be used => "ecmwf"
     *** Cool-skin & Warm-layer schemes will NOT be used!
     *** Computational domain shape: Ni x Nj = 03600 x 02700
     *** Number of time records that will be treated:           1
     *** Number of iterations in bulk algos: nb_iter  =    6
     *** Filling the `mask` array...
         ==> number of points to mask:      2980707  (out of      9720000 )
     *** Type of prescribed air humidity  `specific humidity [kg/kg]`
                    ----- AeroBulk_bye -----
 


2022-06-16 20:49:49,416 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-06-16 20:49:49,418 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-06-16 20:49:49,420 - distributed.nanny - ERROR - Worker process died unexpectedly

KeyboardInterrupt



In [ ]:
ds_coarsened

In [ ]:
# ds_coarsened.to_netcdf('test_coarsened.nc')

## implement a range check

We might want to implement this as an option on the nb/xarray wrapper?

In [ ]:
sst = ds_coarsened.surface_temp + 273.15
t_zt = ds_coarsened.t_ref
hum_zt = ds_coarsened.q_ref
u_zu = ds_coarsened.u_ref
v_zu = ds_coarsened.v_ref
slp = ds_coarsened.slp * 100 # check this


import numpy as np


def _range_check(data, data_min, data_max):
    #fill nans with min val
    data = data.fillna(data_min)
    return np.logical_and(data>=data_min,data<data_max)
    

def check_input_range(sst, t_zt, hum_zt, u_zu, v_zu, slp):
    assert _range_check(sst, 270, 320).all()
    assert _range_check(t_zt, 180, 330).all()
    assert _range_check(hum_zt, 0, 0.08).all()
    assert _range_check(abs(u_zu), 0, 50).all()
    assert _range_check(abs(v_zu), 0, 50).all()
    assert _range_check(slp, 80000, 110000).all()
    
    # additionally check that no nans are in any field except sst?
    assert ~(np.isnan(t_zt).any())
    assert ~(np.isnan(hum_zt).any())
    assert ~(np.isnan(u_zu).any())
    assert ~(np.isnan(v_zu).any())
    assert ~(np.isnan(slp).any())
    
check_input_range(sst, t_zt, hum_zt, u_zu, v_zu, slp)

In [ ]:
ds_coarse_res = noskin_ds_wrapper(ds_coarsened)

In [ ]:
ds_coarsened_mean = ds_coarsened.mean('time')
ds_small_scale_diff = ds_coarsened_mean - ds_coarse_res.mean('time')

In [ ]:
for var in ['ql', 'qh', 'evap']:
    plt.figure()
    plot_ds = ds_small_scale_diff / ds_coarsened_mean *100#np.sign(ds_coarsened_mean)
    plot_ds = plot_ds.where(~coarsened_landmask)
    
    plot_ds[var].plot(robust=True, x='xt_ocean', size=6, aspect=2)

2022-06-16 20:49:49,571 - distributed.nanny - WARNING - Restarting worker
2022-06-16 20:49:49,617 - distributed.nanny - WARNING - Restarting worker
2022-06-16 20:49:49,621 - distributed.nanny - WARNING - Restarting worker
2022-06-16 20:49:49,703 - distributed.nanny - WARNING - Restarting worker
